![This is an image](Quant-Trading.jpg)

<font size="3">
Please visit our website <a href="https://www.quant-trading.co" target="_blank">quant-trading.co</a> for more tools on quantitative finance and data science.
</font>

# **HOW TO MAKE A DYNAMIC CHART FOR THE EVOLUTION OF STOCK RETURNS**

## **Dynamic chart**

<font size="3"> Python allows us to create dynamic charts. This means that we can create kind of videos with financial data. An interesting application of this type of charts is to visualize how financial returns evolve through time.<br><br> 
    
<font size="3"> In this notebook we will use Yahoo Finance Data to make a dynamic chart for the returns of 3 different stocks. 
<br><br>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import datetime
import yfinance as yf


import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Set Plotly to render charts using the web browser, optionally "jupyterlab"
pio.renderers.default = "browser"

## **Download the data**

<font size="3"> We do as usual with the yahoo finance API. Here we are getting 3 years of data for the APPLE, AMAZON and NVIDIA stocks. Then we calculate the total returns as we explained in a previous <a href="https://quant-trading.co/how-to-calculate-cumulative-returns-using-python/" target="_blank">notebook</a>. And after that we modify a little bit the DataFrame to have it in an appropiate format.
<br><br>

In [2]:
#ASSET
tickers_list = ['AAPL', 'AMZN','NVDA']

multiple_prices = yf.download(tickers_list,period='3y')['Adj Close']


for elem in tickers_list:    
    multiple_prices['Return '+ elem] = multiple_prices[elem]/multiple_prices[elem].shift(1) - 1
    multiple_prices['Return '+ elem].iloc[0] = 0

[*********************100%***********************]  3 of 3 completed


In [3]:
multiple_returns = multiple_prices[['Return '+tickers_list[0],'Return '+tickers_list[1],'Return '+tickers_list[2]]]
multiple_returns2 = multiple_returns.copy()

for elem in tickers_list:    
    multiple_returns2['Cum_Return '+ elem] = (1 + multiple_returns['Return '+ elem]).cumprod() - 1

multiple_returns2.reset_index(inplace=True)
multiple_returns2.head(5)

,Date,Return AAPL,Return AMZN,Return NVDA,Cum_Return AAPL,Cum_Return AMZN,Cum_Return NVDA
0,2021-05-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2021-05-11,-0.007410,0.010475,0.002839,-0.007410,0.010475,0.002839
2,2021-05-12,-0.024938,-0.022324,-0.038288,-0.032164,-0.012083,-0.035557
3,2021-05-13,0.017920,0.003024,-0.006778,-0.014821,-0.009096,-0.042094
4,2021-05-14,0.019845,0.019431,0.042279,0.004730,0.010158,-0.001595


In [4]:
multiple_returns2['Date'] = multiple_returns2['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
multiple_returns2['Date'] = pd.to_datetime(multiple_returns2['Date'])
multiple_returns2.head(5)

,Date,Return AAPL,Return AMZN,Return NVDA,Cum_Return AAPL,Cum_Return AMZN,Cum_Return NVDA
0,2021-05-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2021-05-11,-0.007410,0.010475,0.002839,-0.007410,0.010475,0.002839
2,2021-05-12,-0.024938,-0.022324,-0.038288,-0.032164,-0.012083,-0.035557
3,2021-05-13,0.017920,0.003024,-0.006778,-0.014821,-0.009096,-0.042094
4,2021-05-14,0.019845,0.019431,0.042279,0.004730,0.010158,-0.001595


In [5]:
df_returns = multiple_returns2[['Date','Cum_Return '+tickers_list[0],'Cum_Return '+tickers_list[1],'Cum_Return '+tickers_list[2]]]
df_returns.head(5)

,Date,Cum_Return AAPL,Cum_Return AMZN,Cum_Return NVDA
0,2021-05-10,0.000000,0.000000,0.000000
1,2021-05-11,-0.007410,0.010475,0.002839
2,2021-05-12,-0.032164,-0.012083,-0.035557
3,2021-05-13,-0.014821,-0.009096,-0.042094
4,2021-05-14,0.004730,0.010158,-0.001595


## **Melt the DataFrame**

<font size="3"> Here we use the melt method from the pandas library. That allows us to have the data with a single field which we called value and that contains the cumulative returns for the different dates. You can observe that in the following script.
<br><br>

In [6]:
df_returns2 = pd.melt(df_returns, id_vars='Date', value_vars=['Cum_Return '+tickers_list[0], 'Cum_Return '+tickers_list[1], 'Cum_Return '+tickers_list[2]])
df_returns2.head(5)

,Date,variable,value
0,2021-05-10,Cum_Return AAPL,0.000000
1,2021-05-11,Cum_Return AAPL,-0.007410
2,2021-05-12,Cum_Return AAPL,-0.032164
3,2021-05-13,Cum_Return AAPL,-0.014821
4,2021-05-14,Cum_Return AAPL,0.004730


## **Assign names for each stock**

<font size="3"> Because we used the melt method, we lost the name of each stock which were stored in the columns. Therefore, we need to assign the names again. We can do that with the following script.
<br><br>

In [7]:
df_returns2['stock'] = np.nan


conditions = [ (df_returns2['variable'].str.contains(tickers_list[0])) , 
               (df_returns2['variable'].str.contains(tickers_list[1])) , 
               (df_returns2['variable'].str.contains(tickers_list[2]))               
               ]     


choices = [tickers_list[0], tickers_list[1], tickers_list[2]]

df_returns2['stock'] = np.select(conditions, choices, default='null')
df_returns2 = df_returns2.sort_values(['Date', 'stock'], ignore_index=True)
df_returns2.head(5)

,Date,variable,value,stock
0,2021-05-10,Cum_Return AAPL,0.000000,AAPL
1,2021-05-10,Cum_Return AMZN,0.000000,AMZN
2,2021-05-10,Cum_Return NVDA,0.000000,NVDA
3,2021-05-11,Cum_Return AAPL,-0.007410,AAPL
4,2021-05-11,Cum_Return AMZN,0.010475,AMZN


## **Define some parameters**

<font size="3"> Next we define two parameters.<br><br>
    - The first one is the group of colors that we are going to use for the chart.<br><br>
    - The second one one is the number of stocks that we are going to use. In this case, we are using only 3 stocks.
<br><br>

In [8]:
# Adjust colors according to your preference
STOCK_GROUP_COLORS = ['darkblue', 'royalblue', 'dodgerblue'] 
N_UNIQUE_STOCK_GROUPS = df_returns2['stock'].nunique()

## **Create 2 new DataFrames**

<font size="3"> What we are about to do is a small trick to be able to show the data as a movie. We have the cumulative returns for the past 3 years, and all of them are in a single DataFrame. Now we are going to create a new DataFrame that contains a new column which will be called frame. Frame contains the cumulative returns up to a specific date. The first Frame contains only one date, which is the first date on our data sample. The last frame contains all the history.<br><br>
    
<font size="3"> To do that we are going to iterate through all the dates and all the stocks in our DataFrame as follows. You can observe that at the end we will get the df_indexed DataFrame which is the one we will use for our charts<br><br>


In [9]:
df_indexed = pd.DataFrame()

for index in np.arange(start=0,
                       stop=len(df_returns2)+1,
                       step=N_UNIQUE_STOCK_GROUPS):
    df_slicing = df_returns2.iloc[:index].copy()
    df_slicing['frame'] = (index//N_UNIQUE_STOCK_GROUPS)
    df_indexed = pd.concat([df_indexed, df_slicing])

df_slicing is an intermediate step to built the df_indexed DataFrame. Here, the last observations have the total number of frames that we are going to use. 

In [10]:
df_slicing.tail(3)

,Date,variable,value,stock,frame
2265,2024-05-09,Cum_Return AAPL,0.477330,AAPL,756
2266,2024-05-09,Cum_Return AMZN,0.190384,AMZN,756
2267,2024-05-09,Cum_Return NVDA,5.241118,NVDA,756


In [11]:
df_indexed.head(12)

,Date,variable,value,stock,frame
0,2021-05-10,Cum_Return AAPL,0.000000,AAPL,1
1,2021-05-10,Cum_Return AMZN,0.000000,AMZN,1
2,2021-05-10,Cum_Return NVDA,0.000000,NVDA,1
0,2021-05-10,Cum_Return AAPL,0.000000,AAPL,2
1,2021-05-10,Cum_Return AMZN,0.000000,AMZN,2
2,2021-05-10,Cum_Return NVDA,0.000000,NVDA,2
3,2021-05-11,Cum_Return AAPL,-0.007410,AAPL,2
4,2021-05-11,Cum_Return AMZN,0.010475,AMZN,2
5,2021-05-11,Cum_Return NVDA,0.002839,NVDA,2
0,2021-05-10,Cum_Return AAPL,0.000000,AAPL,3


## **Create a scatter plot**

<font size="3"> We will use the library ploty to create these charts.In this case we will use the scatter chart on the df_indexed DataFrame that we already explained. Here is an example of how this chart looks like. We named it scatter_plot_initial, but that's not the one we are using at the end. Is just for illustration purposes<br><br>

In [12]:
scatter_plot_initial = px.scatter(
    df_indexed,
    x='Date',
    y='value',
    color='stock',    
    color_discrete_sequence=STOCK_GROUP_COLORS
)
# scatter_plot_initial.show()

## **Using the animation frame parameter**

<font size="3"> The scatter plot in plotly has the parameter animation_frame. That parameter allows us to create a movie using the field frame that we created some steps above. We named this chart scatter_plot and is the one we are using at the end in this example <br><br>

In [13]:
scatter_plot = px.scatter(
    df_indexed,
    x='Date',
    y='value',
    color='stock',
    animation_frame='frame',
    color_discrete_sequence=STOCK_GROUP_COLORS
)

<font size="3"> We can observe the information contained in each frame from the object scatter_plot below. It is a dictionary with different information. Each frame contains information of the times series for each stock. You can see that the first frame has only one date, the second one two dates and so on<br><br>

In [14]:
scatter_plot.frames[0:1]

(Frame({
     'data': [{'hovertemplate': 'stock=AAPL<br>frame=1<br>Date=%{x}<br>value=%{y}<extra></extra>',
               'legendgroup': 'AAPL',
               'marker': {'color': 'darkblue', 'symbol': 'circle'},
               'mode': 'markers',
               'name': 'AAPL',
               'orientation': 'v',
               'showlegend': True,
               'type': 'scatter',
               'x': array([datetime.datetime(2021, 5, 10, 0, 0)], dtype=object),
               'xaxis': 'x',
               'y': array([0.]),
               'yaxis': 'y'},
              {'hovertemplate': 'stock=AMZN<br>frame=1<br>Date=%{x}<br>value=%{y}<extra></extra>',
               'legendgroup': 'AMZN',
               'marker': {'color': 'royalblue', 'symbol': 'circle'},
               'mode': 'markers',
               'name': 'AMZN',
               'orientation': 'v',
               'showlegend': True,
               'type': 'scatter',
               'x': array([datetime.datetime(2021, 5, 10, 0, 0)], dty

<font size="3"> If we select the first frame, using the field data. 

In [15]:
scatter_plot.frames[0]['data']

(Scatter({
     'hovertemplate': 'stock=AAPL<br>frame=1<br>Date=%{x}<br>value=%{y}<extra></extra>',
     'legendgroup': 'AAPL',
     'marker': {'color': 'darkblue', 'symbol': 'circle'},
     'mode': 'markers',
     'name': 'AAPL',
     'orientation': 'v',
     'showlegend': True,
     'x': array([datetime.datetime(2021, 5, 10, 0, 0)], dtype=object),
     'xaxis': 'x',
     'y': array([0.]),
     'yaxis': 'y'
 }),
 Scatter({
     'hovertemplate': 'stock=AMZN<br>frame=1<br>Date=%{x}<br>value=%{y}<extra></extra>',
     'legendgroup': 'AMZN',
     'marker': {'color': 'royalblue', 'symbol': 'circle'},
     'mode': 'markers',
     'name': 'AMZN',
     'orientation': 'v',
     'showlegend': True,
     'x': array([datetime.datetime(2021, 5, 10, 0, 0)], dtype=object),
     'xaxis': 'x',
     'y': array([0.]),
     'yaxis': 'y'
 }),
 Scatter({
     'hovertemplate': 'stock=NVDA<br>frame=1<br>Date=%{x}<br>value=%{y}<extra></extra>',
     'legendgroup': 'NVDA',
     'marker': {'color': 'dodgerblue'

<font size="3">We can see that for x we have the first date and for y we have the first cumulative return of the first stock.

In [16]:
print(scatter_plot.frames[0]['data'][0]['x'])
print(scatter_plot.frames[0]['data'][0]['y'])

[datetime.datetime(2021, 5, 10, 0, 0)]
[0.]


<font size="3">If we take the 3rd frame, we will have the time series of the dates and the returns of the first stock.

In [17]:
print(scatter_plot.frames[3]['data'][0]['x'])
print(scatter_plot.frames[3]['data'][0]['y'])

[datetime.datetime(2021, 5, 10, 0, 0) datetime.datetime(2021, 5, 11, 0, 0)
 datetime.datetime(2021, 5, 12, 0, 0) datetime.datetime(2021, 5, 13, 0, 0)]
[ 0.         -0.00741023 -0.03216389 -0.01482053]


<font size="3">And here we can observe the same for the second stock

In [18]:
print(scatter_plot.frames[3]['data'][1]['x'])
print(scatter_plot.frames[3]['data'][1]['y'])

[datetime.datetime(2021, 5, 10, 0, 0) datetime.datetime(2021, 5, 11, 0, 0)
 datetime.datetime(2021, 5, 12, 0, 0) datetime.datetime(2021, 5, 13, 0, 0)]
[ 0.          0.01047482 -0.01208282 -0.00909581]


<font size="3">You can corroborate the information from the original DataFrame as shown below

In [19]:
df_returns2.head(12)

,Date,variable,value,stock
0,2021-05-10,Cum_Return AAPL,0.000000,AAPL
1,2021-05-10,Cum_Return AMZN,0.000000,AMZN
2,2021-05-10,Cum_Return NVDA,0.000000,NVDA
3,2021-05-11,Cum_Return AAPL,-0.007410,AAPL
4,2021-05-11,Cum_Return AMZN,0.010475,AMZN
5,2021-05-11,Cum_Return NVDA,0.002839,NVDA
6,2021-05-12,Cum_Return AAPL,-0.032164,AAPL
7,2021-05-12,Cum_Return AMZN,-0.012083,AMZN
8,2021-05-12,Cum_Return NVDA,-0.035557,NVDA
9,2021-05-13,Cum_Return AAPL,-0.014821,AAPL


<font size="3"> Now we can iterate through plot’s properties: data, frames, and layout. Here comes the magic trick: instead of using all data records, update the array and keep only the last value from each iteration using np.take() function. This applies only to the scatter plot. Numpy's take(~) method is used to access values, rows and columns of an array. Here an example on how it works

In [20]:
a = [[4,5,6,9]]
np.take(a, -1)

9

<font size="3"> Now we apply it to our data taking only the last part of the array

In [21]:
for frame in scatter_plot.frames:
    for data in frame.data:
        data.update(mode='markers',
                    showlegend=True,
                    opacity=1)
        data['x'] = np.take(data['x'], [-1])
        data['y'] = np.take(data['y'], [-1])

## **Line plot**

<font size="3"> Now we built a line plot. Here is an example of what we get. We named it line_plot_initial but it is not the one we are using at the end. Is just for illustration purposes. <br><br>

In [22]:
line_plot_initial = px.line(
    df_indexed,
    x='Date',
    y='value',
    color='stock',
    animation_frame='frame',
    color_discrete_sequence=STOCK_GROUP_COLORS,
    width=1000,
    height=500,
    line_shape='spline' # Make a line graph curvy
)
# Hide line plot legend to avoid duplication with scatter plot's legend
line_plot_initial.update_traces(showlegend=False)  

for frame in line_plot_initial.frames:
    for data in frame.data:
        data.update(mode='lines', opacity=0.8, showlegend=False)


# line_plot_initial.show()

<font size="3"> Let's built the chart line_plot, which is the one we are using at the end. Here we can set the width and height of graph, which will be inherited by combined plot later. Notice, that argument showlegend=False is mentioned here twice, once when graph is initialised, then during each interation.

In [23]:
line_plot = px.line(
    df_indexed,
    x='Date',
    y='value',
    color='stock',
    animation_frame='frame',
    color_discrete_sequence=STOCK_GROUP_COLORS,
    width=1000,
    height=500,
    line_shape='spline' # Make a line graph curvy
)
# Hide line plot legend to avoid duplication with scatter plot's legend
line_plot.update_traces(showlegend=False)  

for frame in line_plot.frames:
    for data in frame.data:
        data.update(mode='lines', opacity=0.8, showlegend=False)

## **Combined plot**

<font size="3"> The last step in creating animated graphs is to join the scatter plot with the line plot. We used Plotly Graph Objects go.Figure module to create a new, combined plot, where layout from the line plot is inherited. Here we will use the zip function to iterate throgh the tuple which contains infor from the scatter_plot and the line_plot.<br><br>
    
<font size="3">The zip() function returns a zip object, which is an iterator of tuples where the first item in each passed iterator is paired together, and then the second item in each passed iterator are paired together etc. If the passed iterables have different lengths, the iterable with the least items decides the length of the new iterator. Here is an example of zip

In [24]:
a = ("John", "Charles", "Mike")
b = ("Jenny", "Christy", "Monica")

c = zip(a, b)


for i in c:
    print(i)

('John', 'Jenny')
('Charles', 'Christy')
('Mike', 'Monica')


In [25]:
combined_plot = go.Figure(
    data=line_plot.data + scatter_plot.data,
    frames=[
        go.Frame(data=line_plot.data + scatter_plot.data, name=scatter_plot.name)
        for line_plot, scatter_plot in zip(line_plot.frames, scatter_plot.frames)
    ],
    layout=line_plot.layout
)

# combined_plot.show()

## **Final adjustments**

<font size="3"> Now, customize the graph to your preferences setting the layout. At the end, set the animation speed. <br><br>

In [27]:
combined_plot.update_yaxes(
    gridcolor='white',
    griddash='dot',
    gridwidth=0.5,
    linewidth=2,
    tickwidth=2
)

combined_plot.update_xaxes(
    title_font=dict(size=16),
    linewidth=2,
    tickwidth=2
)

combined_plot.update_traces(
    line=dict(width=5),
    marker=dict(size=10)) # Play with marker size and line width

combined_plot.update_layout(
    font=dict(size=18),
    yaxis=dict(tickfont=dict(size=16)),
    xaxis=dict(tickfont=dict(size=16)),
    showlegend=True,
    legend=dict(title='Stock'),
    template='simple_white',
    title="<b>Total cumulative returns</b>",
    yaxis_title="<b>Cumulative Return</b>",
    xaxis_title="<b>Date</b>",
    yaxis_showgrid=True,
    xaxis_range=[df_indexed['Date'].min() ,
                 df_indexed['Date'].max() ],
    yaxis_range=[df_indexed['value'].min(),
                 df_indexed['value'].max()*1.25],
    plot_bgcolor='white',
    paper_bgcolor='white',
    title_x=0.5
)

combined_plot['layout'].pop("sliders")
combined_plot.layout.updatemenus[0].buttons[0]['args'][1]['frame']['duration'] = 60
combined_plot.layout.updatemenus[0].buttons[0]['args'][1]['transition']['duration'] = 25
combined_plot.layout.updatemenus[0].buttons[0]['args'][1]['transition']['redraw'] = False
combined_plot.show()

<font size="3"> Many of the ideas explained in this nothebook were taken from <a href="https://blog.stackademic.com/bringing-data-to-life-crafting-animated-timeline-graphs-from-dust-0cbb40ff8737" target="_blank">this link</a> . We would like to thank them and give them the credit for their amazing explanation.

If this content is helpful and you want to make a donation please click on the button below. It helps us maintain this site.

[![paypal](https://www.paypalobjects.com/en_US/i/btn/btn_donateCC_LG.gif)](https://www.paypal.com/cgi-bin/webscr?cmd=_s-xclick&hosted_button_id=29CVY97MEQ9BY)